In [ ]:
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.layers.experimental import preprocessing

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import cv2
import random

from tensorflow.keras.layers import Input, Dense, Flatten, GlobalAveragePooling2D, Dropout, BatchNormalization
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers, regularizers
from tensorflow.keras.metrics import Precision, Recall


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
mean = np.array([0.485, 0.456, 0.406])
std = np.array([0.229, 0.224, 0.225])

In [ ]:
from tensorflow.keras.preprocessing import image_dataset_from_directory

train_dir = "/content/drive/MyDrive/dataset_final/train"
test_dir =  "/content/drive/MyDrive/dataset_final/test"
val_dir = "/content/drive/MyDrive/dataset_final/val"

# Create a normalization layer
normalization_layer = preprocessing.Rescaling(1./255, input_shape=(224, 224, 3))  # Assuming pixel values are in [0, 255]

train_data = image_dataset_from_directory(train_dir, label_mode="categorical",
                                          image_size=(224, 224), batch_size=32,
                                          shuffle=True, seed=42)

# Apply normalization to the train_data
normalized_train_data = train_data.map(lambda x, y: (normalization_layer(x), y))

test_data = image_dataset_from_directory(test_dir, label_mode="categorical",
                                         image_size=(224, 224), batch_size=32,
                                         shuffle=False, seed=42)

# Apply normalization to the test_data
normalized_test_data = test_data.map(lambda x, y: (normalization_layer(x), y))

val_data = image_dataset_from_directory(val_dir, label_mode="categorical",
                                        image_size=(224, 224), batch_size=32,
                                        shuffle=False, seed=42)
# Apply normalization to the val_data
normalized_val_data = val_data.map(lambda x, y: (normalization_layer(x), y))

addModel = tf.keras.applications.EfficientNetV2S(input_shape=(224, 224, 3),
                                                  include_top=False,
                                                  weights='imagenet')


Found 10125 files belonging to 10 classes.
Found 1449 files belonging to 10 classes.
Found 2890 files belonging to 10 classes.
82420632/82420632 [==============================] - 1s 0us/step


In [ ]:

model = Sequential()
model.add(addModel)
model.add(GlobalAveragePooling2D())

model.add(Flatten())
model.add(Dense(256, activation="relu"))

model.add(Dense(128, activation="relu"))


model.add(Dense(10, activation="softmax" , name="classification"))

In [ ]:
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
            loss='categorical_crossentropy',
            metrics=['accuracy'])

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 efficientnetv2-s (Function  (None, 7, 7, 1280)        20331360  
 al)                                                             
                                                                 
 global_average_pooling2d (  (None, 1280)              0         
 GlobalAveragePooling2D)                                         
                                                                 
 flatten (Flatten)           (None, 1280)              0         
                                                                 
 dense (Dense)               (None, 256)               327936    
                                                                 
 dense_1 (Dense)             (None, 128)               32896     
                                                                 
 classification (Dense)      (None, 10)                1

In [ ]:
history_1 = model.fit(train_data , validation_data = val_data , epochs = 12)

Epoch 1/12
317/317 [==============================] - 2987s 9s/step - loss: 1.6752 - accuracy: 0.4190 - val_loss: 1.5096 - val_accuracy: 0.4747
Epoch 2/12
317/317 [==============================] - 177s 555ms/step - loss: 1.4006 - accuracy: 0.5203 - val_loss: 1.4000 - val_accuracy: 0.5401
Epoch 3/12
317/317 [==============================] - 183s 574ms/step - loss: 1.2181 - accuracy: 0.5851 - val_loss: 1.3417 - val_accuracy: 0.5682
Epoch 4/12
317/317 [==============================] - 175s 549ms/step - loss: 1.0931 - accuracy: 0.6310 - val_loss: 1.2726 - val_accuracy: 0.5910
Epoch 5/12
317/317 [==============================] - 176s 552ms/step - loss: 0.9488 - accuracy: 0.6781 - val_loss: 1.4223 - val_accuracy: 0.5782
Epoch 6/12
317/317 [==============================] - 175s 549ms/step - loss: 0.8442 - accuracy: 0.7146 - val_loss: 1.3057 - val_accuracy: 0.6107
Epoch 7/12
317/317 [==============================] - 175s 549ms/step - loss: 0.7073 - accuracy: 0.7597 - val_loss: 1.3941 - v

In [ ]:
model.save('/content/drive/MyDrive/skin_ultra.h5')

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [ ]:
model_save_path = "/content/drive/MyDrive/saved_models/skin_disease_model_70"

model.save(model_save_path)

print("Model saved successfully at:", model_save_path)


Model saved successfully at: /content/drive/MyDrive/saved_models/skin_disease_model_70


In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing import image
import numpy as np

# Load the saved model
model_save_path = "/content/drive/MyDrive/saved_models/skin_disease_model"
loaded_model = tf.keras.models.load_model(model_save_path)

# Assuming you have a separate test dataset (test_data) for evaluation
test_loss, test_accuracy = loaded_model.evaluate(test_data)
print(f"\nTest Accuracy: {test_accuracy*100:.2f}%")

# Calculate top-3 accuracy manually
top3_predictions = np.argsort(loaded_model.predict(test_data), axis=1)[:, -3:]  # Get indices of top 3 predictions
true_labels = np.concatenate([y for x, y in test_data], axis=0)  # Assuming your test_data is a tuple (images, labels)

top3_correct = np.any(top3_predictions == np.argmax(true_labels, axis=1).reshape(-1, 1), axis=1)
top3_accuracy = np.mean(top3_correct)

print(f"\nTop-3 Accuracy: {top3_accuracy*100:.2f}%")

# Calculate top-5 accuracy manually
top5_predictions = np.argsort(loaded_model.predict(test_data), axis=1)[:, -5:]  # Get indices of top 5 predictions

top5_correct = np.any(top5_predictions == np.argmax(true_labels, axis=1).reshape(-1, 1), axis=1)
top5_accuracy = np.mean(top5_correct)

print(f"\nTop-5 Accuracy: {top5_accuracy*100:.2f}%")


46/46 [==============================] - 176s 4s/step - loss: 1.4927 - accuracy: 0.6308

Test Accuracy: 63.08%
46/46 [==============================] - 12s 195ms/step

Top-3 Accuracy: 86.61%
46/46 [==============================] - 9s 196ms/step

Top-5 Accuracy: 94.62%


In [ ]:
from sklearn.metrics import classification_report
import numpy as np

# Assuming you have a separate test dataset without the normalization
test_data = image_dataset_from_directory(test_dir, label_mode="categorical",
                                         image_size=(224, 224), batch_size=32,
                                         shuffle=False, seed=42)

loaded_model = tf.keras.models.load_model("/content/drive/MyDrive/saved_models/skin_disease_model")
# Get true labels and predicted probabilities from the model

true_labels = np.concatenate([y for x, y in test_data], axis=0)
predicted_probs = loaded_model.predict(test_data)

# Convert probabilities to class labels
predicted_labels = np.argmax(predicted_probs, axis=1)

# Define class names
class_names = [
    "Bacterial Infections",
    "Chronic Skin Conditions",
    "Sexually Transmitted Infections",
    "Connective Tissue Disorders",
    "Nail Conditions",
    "Contact Dermatitis",
    "Infestations and Bites",
    "Fungal Infections",
    "Viral Infections",
    "Miscellaneous Skin Conditions"
]

# Use the classification_report to get precision, recall, and F1-score for each class
report = classification_report(np.argmax(true_labels, axis=1), predicted_labels, target_names=class_names)

print("Classification Report:\n", report)



Found 1449 files belonging to 10 classes.
46/46 [==============================] - 11s 180ms/step
Classification Report:
                                  precision    recall  f1-score   support

           Bacterial Infections       0.63      0.71      0.67       153
        Chronic Skin Conditions       0.65      0.71      0.68       251
Sexually Transmitted Infections       0.62      0.51      0.56       108
    Connective Tissue Disorders       0.61      0.49      0.55        97
                Nail Conditions       0.68      0.66      0.67       103
             Contact Dermatitis       0.69      0.60      0.64       141
         Infestations and Bites       0.53      0.58      0.55       274
              Fungal Infections       0.85      0.91      0.88       110
               Viral Infections       0.49      0.53      0.51        66
  Miscellaneous Skin Conditions       0.62      0.53      0.58       146

                       accuracy                           0.63      1449
